In [2]:
import os
import cv2
from glob import glob
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.layers import Input, Dense, Conv2D,Flatten, LeakyReLU, BatchNormalization, Add, UpSampling2D
from tensorflow.keras.models import Model


In [3]:

# Paths to your dataset
base_dir = 'images'  # Replace with the path to your dataset directory
hr_train_dir = os.path.join(base_dir, 'train')
hr_val_dir = os.path.join(base_dir, 'val')
hr_test_dir = os.path.join(base_dir, 'test')

# Directories to save low-resolution images
lr_train_dir = os.path.join(base_dir, 'train_LR')
lr_val_dir = os.path.join(base_dir, 'val_LR')
lr_test_dir = os.path.join(base_dir, 'test_LR')

# Ensure LR directories exist
os.makedirs(lr_train_dir, exist_ok=True)
os.makedirs(lr_val_dir, exist_ok=True)
os.makedirs(lr_test_dir, exist_ok=True)


In [4]:

# Function to downsample images
def downsample_image(image, scale=4):
    height, width = image.shape[:2]
    lr_image = cv2.resize(image, (width // scale, height // scale), interpolation=cv2.INTER_CUBIC)
    return lr_image

# Process and save LR images
def create_lr_images(hr_dir, lr_dir, scale=4):
    for filepath in glob(os.path.join(hr_dir, "*.jpg")):
        img = cv2.imread(filepath)
        if img is not None:
            lr_img = downsample_image(img, scale=scale)
            lr_filepath = os.path.join(lr_dir, os.path.basename(filepath))
            cv2.imwrite(lr_filepath, lr_img)


In [5]:

# Create LR images for train, validation, and test sets
create_lr_images(hr_train_dir, lr_train_dir)
create_lr_images(hr_val_dir, lr_val_dir)
create_lr_images(hr_test_dir, lr_test_dir)

print("Low-resolution images created.")

def load_image_paths(lr_dir, hr_dir):
    lr_images = sorted(glob(os.path.join(lr_dir, '*.jpg')))
    hr_images = sorted(glob(os.path.join(hr_dir, '*.jpg')))
    return lr_images, hr_images

# Load image paths for train, validation, and test sets
lr_train_images, hr_train_images = load_image_paths(lr_train_dir, hr_train_dir)
lr_val_images, hr_val_images = load_image_paths(lr_val_dir, hr_val_dir)
lr_test_images, hr_test_images = load_image_paths(lr_test_dir, hr_test_dir)

def preprocess_image(image_path, target_size=(16, 16)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, target_size)  # Resize to match the expected input size
    img = img / 127.5 - 1  # Normalize to [-1, 1] for GANs
    return img

def data_generator(lr_image_paths, hr_image_paths, batch_size, target_size):
    while True:
        lr_batch = []
        hr_batch = []

        # Randomly select `batch_size` images for each batch
        selected_indices = np.random.randint(0, len(lr_image_paths), batch_size)
        
        for idx in selected_indices:
            # Load and preprocess the low-resolution and high-resolution images
            lr_img = load_and_preprocess_image(lr_image_paths[idx], target_size)
            hr_img = load_and_preprocess_image(hr_image_paths[idx], (64, 64))  # Resize to match the discriminator's input
            
            lr_batch.append(lr_img)
            hr_batch.append(hr_img)
        
        yield np.array(lr_batch), np.array(hr_batch)

def load_and_preprocess_image(image_path, target_size):
    """Loads an image from a path and resizes it to the target size."""
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return img


Low-resolution images created.


In [6]:
# Adjusted Generator with upsampling and matching skip connections
def build_generator(input_shape=(16, 16, 3)):
    input_img = Input(shape=input_shape)

    # Initial convolutional block
    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = LeakyReLU(alpha=0.2)(x)

    # Downsampling block
    for _ in range(2):
        x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)  # Downsamples by a factor of 2
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)

    # Save intermediate output for skip connection
    skip_connection = x

    # Upsampling blocks to reach (64, 64, 3)
    for _ in range(4):  # Upsample four times to achieve the target resolution
        x = UpSampling2D()(x)
        x = Conv2D(128, (3, 3), padding='same')(x)
        x = LeakyReLU(alpha=0.2)(x)

    # Resize skip connection to match output dimensions
    skip_connection = UpSampling2D(size=(16, 16))(skip_connection)  # Upsample to (64, 64)
    skip_connection = Conv2D(128, (3, 3), padding='same')(skip_connection)

    # Combine the upsampled output and skip connection
    x = Add()([x, skip_connection])

    # Final convolutional layer to generate the output image
    output = Conv2D(3, kernel_size=3, strides=1, padding='same', activation='tanh')(x)
    model = Model(input_img, output)
    return model

# Adjusted Discriminator for (64, 64, 3) input shape
def build_discriminator(input_shape=(64, 64, 3)):
    input_image = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(input_image)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)

    output = Dense(1, activation='sigmoid')(x)
    model = Model(input_image, output)
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    input_image = Input(shape=(16, 16, 3))  # Input shape matches the low-resolution image
    generated_image = generator(input_image)
    valid = discriminator(generated_image)
    model = Model(input_image, valid)
    return model

# Initialize models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Compile the discriminator and GAN
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')


In [7]:
generator.summary()
discriminator.summary()
gan.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 16, 16, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 64)           1792      ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 16, 16, 64)           0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 64)             36928     ['leaky_re_lu[0][0]']         
                                                                                              

In [8]:
# Training parameters
epochs = 10000  # Number of epochs
batch_size = 16  # Batch size
save_interval = 500  # Interval to save model weights
target_size = (16, 16)  # Target size for input images

# Load training data
lr_train_images, hr_train_images = load_image_paths(lr_train_dir, hr_train_dir)
train_generator = data_generator(lr_train_images, hr_train_images, batch_size, target_size)

# Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Training loop
for epoch in range(epochs):
    # Get a batch of real images and corresponding low-resolution images
    lr_imgs, hr_imgs = next(train_generator)
    
    # Generate high-resolution images from low-resolution input
    gen_imgs = generator.predict(lr_imgs)
    
    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(hr_imgs, real)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train the generator (through the combined model)
    g_loss = gan.train_on_batch(lr_imgs, real)
    
    # Print the progress at specified intervals
    if epoch % 100 == 0:
        print(f"Epoch {epoch + 1}/{epochs} | D Loss: {d_loss[0]:.4f}, D Acc: {d_loss[1]:.4f} | G Loss: {g_loss:.4f}")
    
    # Save models at specified intervals
    if (epoch + 1) % save_interval == 0:
        generator.save(f"generator_epoch_{epoch + 1}.h5")
        discriminator.save(f"discriminator_epoch_{epoch + 1}.h5")
        print(f"Saved models at epoch {epoch + 1}")

# Save the final trained models after training completes
generator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
generator.save("generator_final.h5")
discriminator.save("discriminator_final.h5")
print("Training completed and final models saved.")


1/1 [==============================] - 0s 136ms/step
Epoch 1/10000 | D Loss: 0.8699, D Acc: 0.5000 | G Loss: 0.4084
1/1 [==============================] - 0s 270ms/step
Epoch 101/10000 | D Loss: 0.8796, D Acc: 0.5000 | G Loss: 0.4064
1/1 [==============================] - 0s 288ms/step
Epoch 201/10000 | D Loss: 0.8878, D Acc: 0.5000 | G Loss: 0.3987
1/1 [==============================] - 0s 259ms/step
Epoch 301/10000 | D Loss: 0.8921, D Acc: 0.5000 | G Loss: 0.3961
1/1 [==============================] - 0s 280ms/step
Epoch 401/10000 | D Loss: 0.9006, D Acc: 0.5000 | G Loss: 0.3864
1/1 [==============================] - 0s 425ms/step


e:\anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved models at epoch 500
1/1 [==============================] - 0s 314ms/step
Epoch 501/10000 | D Loss: 0.9028, D Acc: 0.5000 | G Loss: 0.3862
1/1 [==============================] - 0s 322ms/step
Epoch 601/10000 | D Loss: 0.9099, D Acc: 0.5000 | G Loss: 0.3800
1/1 [==============================] - 0s 298ms/step
Epoch 701/10000 | D Loss: 0.9166, D Acc: 0.5000 | G Loss: 0.3688
1/1 [==============================] - 0s 450ms/step
Epoch 801/10000 | D Loss: 0.9269, D Acc: 0.5000 | G Loss: 0.3623
1/1 [==============================] - 0s 279ms/step
Epoch 901/10000 | D Loss: 0.9293, D Acc: 0.5000 | G Loss: 0.3617
1/1 [==============================] - 0s 300ms/step
Saved models at epoch 1000
1/1 [==============================] - 0s 273ms/step
Epoch 1001/10000 | D Loss: 0.9344, D Acc: 0.5000 | G Loss: 0.3557
1/1 [==============================] - 0s 259ms/step
Epoch 1101/10000 | D Loss: 0.9438, D Acc: 0.5000 | G Loss: 0.3485
1/1 [==============================] - 0s 288ms/step
Epoch 1201/10